### Basic Verlet Algorithm for Molecular Simulation

### Importing the necessary modules 

In [13]:
import numpy as np
from matplotlib.pylab import *
%matplotlib inline
import vpython as vp
from vpython import *
scene = vp.canvas()

<IPython.core.display.Javascript object>

### Defining the function for uniform random numbers 

In [ ]:
def crand(seed = 1345678,up = 19,low = 1):
    #seed = 1345678
    a = 11
    m = 10000
    l = []
    for i in range(100):
        seed = (seed*(a**i))%m
        l.append(((seed/m)*(up-low))+low)
    np.random.shuffle(l)
    b = l[:3]
    return b

### Defining a function for distances

In [ ]:
def dist_tp(l1,l2):
    dx = l1[0] - l2[0]
    dy = l1[1] - l2[1]
    dz = l1[2] - l2[2]
    dx = dx - Lx*int(round((1.0*dx)/Lx))
    dy = dy - Ly*int(round((1.0*dy)/Ly))
    dz = dz - Lz*int(round((1.0*dz)/Lz))
    return(dx**2+dy**2+dz**2)

### Function for calculating the force using lennard Jones Potential

In [ ]:
def f(dx,dy,dz):
    f1 = 48*((dx**2+dy**2+dz**2)**(-14) - 0.5*(dx**2+dy**2+dz**2)**(-8))*dx
    f2 = 48*((dx**2+dy**2+dz**2)**(-14) - 0.5*(dx**2+dy**2+dz**2)**(-8))*dy
    f3 = 48*((dx**2+dy**2+dz**2)**(-14) - 0.5*(dx**2+dy**2+dz**2)**(-8))*dz
    return(f1,f2,f3)

### Initializing the initial positions to the particles 

In [ ]:
Lx = 20
Ly = 20
Lz = 20 
sigma = 2
t = 0 
nop = 100
rc = (sigma**(1.0/6))*sigma

### Assigning the positions to the particles one by one 

In [ ]:
pos = []
pos.append(crand())
for i in range(1,nop):
    dist = []
    l = crand() 
    for j in range(i):
        dist.append(dist_tp(l,pos[j]))
    while min(dist) <= 4:
        dist = []
        l = crand()
        for j in range(i):
            dist.append(dist_tp(l,pos[j]))
    pos.append(l)

 ### Assigning initial velocities between -1 and 1

In [ ]:
velocity = []
vx = []
vy = []
vz = []
for i in range(nop):
    velocity.append(crand(up=1.0,low=-1.0))    
    vx.append(velocity[i][0])
    vy.append(velocity[i][1])
    vz.append(velocity[i][2]) 

### Scaling momentum to zero

In [ ]:
px = 0
py = 0 
pz = 0 
for i in range(nop):
    px += vx[i]
    py += vy[i]
    pz += vz[i]   
for i in range(nop):
    vx[i] -= (px/nop)
    vy[i] -= (py/nop)
    vz[i] -= (pz/nop)   

### Velocity Rescaling

In [ ]:
T_init = 300
temp = sum(vx[i]**2 + vy[i]**2 + vz[i]**2 for i in range(nop))
temp /= (3*nop)
alpha = (T_init/temp)**0.5
for i in range(nop):
    vx[i] *= alpha
    vy[i] *= alpha
    vz[i] *= alpha
temp = sum(vx[i]**2 + vy[i]**2 + vz[i]**2 for i in range(nop))
temp /= (3*nop) 

### Declaring the variables and lists for storing different parameters of the program

In [ ]:
t = 0 ; tmax = 1 ; nt = 500 ; dt = float(tmax/nt)
px = 0; py = 0; pz = 0; P1a = [] ; P2a = [] 
rx = np.array([pos[i][0] for i in range(nop)]); ry = np.array([pos[i][1] for i in range(nop)]); rz = np.array([pos[i][2] for i in range(nop)])
rox = np.zeros(nop); roy = np.zeros(nop); roz = np.zeros(nop)
rnx = np.zeros(nop); rny = np.zeros(nop); rnz = np.zeros(nop)
fx = np.zeros(nop); fy = np.zeros(nop); fz = np.zeros(nop)
vnx = np.zeros(nop); vny = np.zeros(nop); vnz = np.zeros(nop)
px1 = []; py1 = []; pz1 =[]; fx1 = []; fy1 = []; fz1 = []; Temp = []

### Calculating the position of the particles at previous time step

In [ ]:
for i in range(nop-1):
    a=0;b=0;c=0
    for j in range(i+1,nop):
        dx = rx[i] - rx[j];dy = ry[i] - ry[j]; dz = rz[i] - rz[j]
        dx = dx - Lx*int(round((1.0*dx)/Lx));dy = dy - Ly*int(round((1.0*dy)/Ly));dz = dz - Lz*int(round((1.0*dz)/Lz))
        r = dx**2+dy**2+dz**2
        if (r**0.5) < rc:
            a,b,c = f(dx,dy,dz)
        fx[i] += a; fx[j] -= a
        fy[i] += b; fy[j] -= b
        fz[i] += c; fz[j] -= c
for i in range(nop):
    rox[i] = rx[i] - vx[i]*dt + 0.5*fx[i]*(dt**2)
    roy[i] = ry[i] - vy[i]*dt + 0.5*fy[i]*(dt**2)
    roz[i] = rz[i] - vz[i]*dt + 0.5*fz[i]*(dt**2)

### Time loop 

In [ ]:

for i in range(nop-1):
    a=0;b=0;c=0
    for j in range(i+1,nop):
        dx = rx[i] - rx[j];dy = ry[i] - ry[j]; dz = rz[i] - rz[j]
        dx = dx - Lx*int(round((1.0*dx)/Lx));dy = dy - Ly*int(round((1.0*dy)/Ly));dz = dz - Lz*int(round((1.0*dz)/Lz))
        r = dx**2+dy**2+dz**2
        if (r**0.5) < rc:
            a,b,c = f(dx,dy,dz)
        fx[i] += a; fx[j] -= a
        fy[i] += b; fy[j] -= b
        fz[i] += c; fz[j] -= c
for i in range(nop):
    rox[i] = rx[i] - vx[i]*dt + 0.5*fx[i]*(dt**2)
    roy[i] = ry[i] - vy[i]*dt + 0.5*fy[i]*(dt**2)
    roz[i] = rz[i] - vz[i]*dt + 0.5*fz[i]*(dt**2)

while t <= tmax:
    c0 = 0;c1=0;c2=0;c3=0;c4=0;c5=0;c6=0;
    P1 = 0; P2 = 0
    fx = np.zeros(nop); fy = np.zeros(nop); fz = np.zeros(nop)
    for i in range(nop-1):
        for j in range(i+1,nop):
            dx = rx[i] - rx[j];dy = ry[i] - ry[j]; dz = rz[i] - rz[j]
            dx = dx - Lx*int(round((1.0*dx)/Lx));dy = dy - Ly*int(round((1.0*dy)/Ly));dz = dz - Lz*int(round((1.0*dz)/Lz))
            r = dx**2+dy**2+dz**2
            if (r**0.5) < rc:
                a,b,c = f(dx,dy,dz)
            fx[i] += a; fx[j] -= a
            fy[i] += b; fy[j] -= b
            fz[i] += c; fz[j] -= c  
            P1+= fx[i]*dx + fy[i]*dy + fz[i]*dz
    for i in range(nop):
        rnx[i] = 2*rx[i] - rox[i] + fx[i]*dt**2
        rny[i] = 2*ry[i] - roy[i] + fy[i]*dt**2
        rnz[i] = 2*rz[i] - roz[i] + fz[i]*dt**2
        if rnx[i] > 50 or rny[i] > 50 or rnz[i] > 50:
            rnx[i] -= Lx
            rny[i] -= Ly
            rnz[i] -= Lz
        if rnx[i] < 0 or rny[i] < 0 or rnz[i] < 0:
            rnx[i] +=Lx
            rny[i] +=Ly
            rnz[i] +=Lz
        vnx[i] = vx[i] + fx[i]*dt
        vny[i] = vy[i] + fy[i]*dt
        vnz[i] = vz[i] + fz[i]*dt
        
    for i in range(nop):
        c0 += vnx[i]**2+vny[i]**2+vnz[i]**2
        c1 += vnx[i];c2 += vny[i];c3 += vnz[i];c4 += fx[i];c5 += fy[i]; c6 += fz[i]
    vol = Lx*Ly*Lz
    nd = nop/(1.0*Lx*Ly*Lz)
    P1/=vol
    P2 = nd*(c0/(3*nop))
    Temp.append(c0/(3*nop)); px1.append(c1); py1.append(c2);pz1.append(c3);fx1.append(c4);fy1.append(c5);fz1.append(c6)      
    P1a.append(P1); P2a.append(P2)
    rox = rx; roy = ry; roz = rz
    rx = rnx; ry = rny; rz = rnz
    t += dt

### Plotting the dynamics of physical parameters of the simulation

In [ ]:
figure(figsize = (18,5))    
tvec = linspace(0,tmax,nt)
xlabel('time(sec)')
ylabel('Temperature(K)')
scatter(tvec,Temp)
show()
#----------------------------
figure(figsize = (20,5))
subplot(1,3,1)
xlabel('time(sec)')
ylabel('Momentum(Px)')
plot(tvec,px1)
subplot(1,3,2)
xlabel('time(sec)')
ylabel('Momentum(Py)')
plot(tvec,py1)
subplot(1,3,3)
xlabel('time(sec)')
ylabel('Momentum(Pz)')
plot(tvec,pz1)
show()
#----------------------------
figure(figsize = (18,5))
subplot(1,3,1)
xlabel('time(sec)')
ylabel('Force(Fx)')
plot(tvec,fx1)
subplot(1,3,2)
xlabel('time(sec)')
ylabel('Force(Fy)')
plot(tvec,fy1)
subplot(1,3,3)
xlabel('time(sec)')
ylabel('Force(Fz)')
plot(tvec,fz1)
show()